In [3]:
# Loading Data

%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import random


# using the SQLite Table to read data.
con = sqlite3.connect('final.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
cleaned_data = pd.read_sql_query("""
SELECT *
FROM Reviews
""", con) 
#select a random 150 rows 

data_pos = cleaned_data[cleaned_data["Score"]=='positive'].sample(n = 5000,replace=True)
data_neg = cleaned_data[cleaned_data["Score"]=='negative'].sample(n = 5000, replace=True)
final_data = pd.concat([data_pos, data_neg])
cleaned_data['Score'].value_counts()

positive    38479
negative     7592
Name: Score, dtype: int64

In [4]:
# Sorting data based on time
# convert time into understnad time
final_data['Time'] = pd.to_datetime(final_data['Time'], unit = 's')
final_data

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
2343,5815,6294,B00061U6AW,A36YY6UTUGYDQF,Jaydenzmommi,0,0,positive,2010-11-27,Yummy!!!,"The actual product deserves 5 stars, the reaso...",b'actual product deserv star reason gave star ...
45507,2247,2442,B0089SPDUW,AVN3JX894CEZC,Zugg Zugg,0,0,positive,2011-04-01,Caribou Coffee's Mahogany Review,The Mahogany has a smooth full bodied characte...,b'mahogani smooth full bodi charact high recom...
7255,25254,27582,B000FK52B2,A39F8VJHN07PUX,Hank,5,5,positive,2010-11-17,Great Item,"I love fresh sprouts, but the problem with my ...",b'love fresh sprout problem old sprouter simpl...
29256,39425,42818,B002NHYQAS,A2P6KAWXJ16234,Kelly (Fantasy Literature),0,0,positive,2011-02-23,"Mmm, chocolate","This chocolate bar arrived today, and I didn't...",b'chocol bar arriv today didnt wait long tri s...
21950,41515,45115,B001EQ5AK4,ASRH8KBNJXZLT,D.H. &quot;Saint&quot;,1,1,positive,2010-01-27,"""The BEST ""","This is the best maple syrup that I have had, ...",b'best mapl syrup long time fact good put seco...
18449,18665,20352,B001BDDTB2,A2DYW9Y2D8E43L,"Kris Cayocca ""Cayotejr""",4,4,positive,2008-11-30,The cat seems pleased.,"The cat went right for this food, no hesitatio...",b'cat went right food hesit rate star can food...
44961,14583,15900,B007TJGZ5E,AJ1H90XS8PLXN,Whipple,0,0,positive,2012-04-04,AWESOME,This worked out great! I ordered 4 boxes! IT w...,b'work great order box great deal top use disc...
26447,10212,11156,B00213ERI0,AARU93A331T4I,elijah's mama,0,0,positive,2012-05-02,My sons is on a gfcf diet,My son is on a restricted diet due to allergie...,b'son restrict diet due allergi picki hard fin...
4317,27694,30191,B000DZFMEQ,A3UXMCZD5HHEE1,Wendy Moffitt,0,0,positive,2007-05-19,happiness in a bag,This product is the BEST!!!! It is so flavorfu...,b'product best flavor rich moist made follow d...
38135,10417,11376,B004CPAPBS,A2T0V99GLGXWCM,Ghost(Ghost(M)),1,1,positive,2011-06-27,"Good prunes, but what's with the price here? E...",I made prunes a permanent part of my diet beca...,b'made prune perman part diet promot best food...


In [0]:
final_data = final_data.sort_values(by='Time')

In [6]:
X = final_data['CleanedText']
print('shape of X:', X.shape)

shape of X: (10000,)


In [7]:
# class label
y = final_data["Score"]
print('shape of y:', y.shape)


shape of y: (10000,)


In [8]:
# split data into train and test where 70% data used to train model and 30% for test
# final_data[:int(len(final_data) * 0.75)], final_data[int(len(final_data) * 0.75):]
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape, y_train.shape, x_test.shape)


(7000,) (7000,) (3000,)


In [9]:
# BOW 
# Train Vectorizor
from sklearn.feature_extraction.text import CountVectorizer 

bow = CountVectorizer()
X_train = bow.fit_transform(X_train)
X_train



<7000x10415 sparse matrix of type '<class 'numpy.int64'>'
	with 225681 stored elements in Compressed Sparse Row format>

In [10]:
# Test Vectorizor

x_test = bow.transform(x_test)
x_test




<3000x10415 sparse matrix of type '<class 'numpy.int64'>'
	with 97429 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn import tree
from sklearn import metrics
clf = tree.DecisionTreeClassifier(max_depth=19)
clf.fit(X_train, y_train)
pred = clf.predict(x_test)
print("Test Accuracy is ",metrics.accuracy_score(y_test, pred))


Test Accuracy is  0.761


In [0]:
from sklearn.model_selection import GridSearchCV
def DecisionTree(X_train, y_train, x_test, y_test):
    
    parameters = {'max_depth' : range(2, 20)}
    clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, cv=5, n_jobs=4)     
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with gridSearchCV is:'+ str(clf.score(x_test, y_test)))

        
        
    

In [13]:
DecisionTree(X_train, y_train, x_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=19,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
The score with gridSearchCV is:0.7556666666666667


In [14]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=400, max_depth = 2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [15]:
pred = clf.predict(x_test)
print("Accuracy of test data :", metrics.accuracy_score(y_test, pred))

Accuracy of test data : 0.7733333333333333


In [16]:
print(clf.feature_importances_)

[0. 0. 0. ... 0. 0. 0.]


In [17]:
#TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
# Split data
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape, x_test.shape, y_train.shape, y_test.shape)
#tfidf = TfidfVectorizer()
#tfidf_data = tfidf.fit_transform(final_4000["CleanedText"])
#tfidf_data
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
X_train = tf_idf_vect.fit_transform(X_train)
X_train

(7000,) (3000,) (7000,) (3000,)


<7000x162833 sparse matrix of type '<class 'numpy.float64'>'
	with 489402 stored elements in Compressed Sparse Row format>

In [18]:
# Convert test text data to its vectorizor
x_test = tf_idf_vect.transform(x_test)
x_test.shape

(3000, 162833)

In [0]:
from sklearn.model_selection import GridSearchCV
def DecisionTree(X_train, y_train, x_test, y_test):
    
    parameters = {'max_depth' : range(2, 20)}
    clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, cv=5, n_jobs=4)     
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with gridSearchCV is:'+ str(clf.score(x_test, y_test)))

        
        
    

In [20]:
DecisionTree(X_train, y_train, x_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=18,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
The score with gridSearchCV is:0.7443333333333333


In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=400, max_depth = 2, random_state=0)
clf.fit(X_train, y_train)
pred = clf.predict(x_test)
print("Accuracy of test data :", metrics.accuracy_score(y_test, pred))

Accuracy of test data : 0.6893333333333334


In [22]:
#W2V

# split data into train and test where 70% data used to train model and 30% for test
# final_data[:int(len(final_data) * 0.75)], final_data[int(len(final_data) * 0.75):]
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape, y_train.shape, x_test.shape)


(7000,) (7000,) (3000,)


In [0]:

import gensim
import re

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    sentence = sentence.decode('utf-8')
    cleantext = re.sub(cleanr, "", sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned


In [0]:
i=0
list_of_sent=[]
for sent in X_train.values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

In [25]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [26]:
w2v = w2v_model[w2v_model.wv.vocab]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [0]:
# Train your own Word2Vec model using your own test text corpus
import gensim
list_of_sent_test = []
#for sent in final_40k['Text'].values:
for sent in x_test:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent_test.append(filtered_sentence)

In [28]:
w2v_model=gensim.models.Word2Vec(list_of_sent_test, min_count=5, size=50, workers=4)

In [29]:
w2v = w2v_model[w2v_model.wv.vocab]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [30]:
# Average Word2Vec
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))
sent_vectors= np.nan_to_num(sent_vectors)

7000
50


In [31]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent_test: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
print(len(sent_vectors_test))
print(len(sent_vectors_test[0]))
sent_vectors_test = np.nan_to_num(sent_vectors_test)

3000
50


In [0]:
X_train = sent_vectors
x_test = sent_vectors_test

In [0]:
from sklearn.model_selection import GridSearchCV
def DecisionTree(X_train, y_train, x_test, y_test):
    
    parameters = {'max_depth' : range(2, 20)}
    clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, cv=7)     
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with gridSearchCV is:'+ str(clf.score(x_test, y_test)))

        
        
    

In [34]:
DecisionTree(X_train, y_train, x_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=19,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
The score with gridSearchCV is:0.6866666666666666


In [0]:
from sklearn.model_selection import GridSearchCV
def RandomForest(X_train, y_train, x_test, y_test):
    
    parameters = {'n_estimators':range(100, 110),'max_depth' : range(2, 20)}
    clf = GridSearchCV(RandomForestClassifier(), parameters, cv=7)     
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with gridSearchCV is:'+ str(clf.score(x_test, y_test)))

        
        
    

In [61]:
%time RandomForest(X_train, y_train, x_test, y_test)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=19, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=108, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
The score with gridSearchCV is:0.7543333333333333
CPU times: user 58min 32s, sys: 658 ms, total: 58min 33s
Wall time: 58min 34s
